<a href="https://colab.research.google.com/github/Cairo-Henrique/Analise-Fundamentalista/blob/main/Value_Investing_with_Financial_indicators_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Value Investing with Financial Indicators Tool**

This notebook provides a systematic framework for evaluating stocks based on **Value Investing** principles. It automates the collection of fundamental data and scores companies against established financial benchmarks to identify potentially undervalued assets with strong balance sheets.

---

## **Methodology**

Each ticker is analyzed using 10 financial indicators. A Score is then generated based on how many of these indicators meet "Good" thresholds:

---

## **Notebook Features**

1. **Automated Data Retrieval:** Pulls `ticker.info` and historical price data for any stock ticker.
2. **Qualitative Assessment:** Automatically labels indicators as "Good" or "Bad" based on fundamentalist thresholds.
3. **Comparative Ranking:** Generates a ranking table for multiple stocks, showing which companies have the highest proportion of healthy indicators.
4. **Performance Visualization:** Plots the historical percentage return of the selected assets to compare fundamental strength with market performance.

---

## **3. Libraries Used**

* `yfinance`: For fetching stock metadata, financial statements, and historical prices.
* `pandas`: For data structuring and ranking logic.
* `matplotlib`: For visualizing asset returns over time.

---

### **How to Use**

1. **Setup:** Run the first cell to import necessary libraries.
2. **Input:** Enter the list of tickers you wish to analyze (e.g., `['AAPL', 'GOOGL', 'MSFT']`.
3. **Analysis:** Observe the printed breakdown for each company and the final ranking table.

In [5]:
import yfinance as yf
import pandas as pd


In [6]:
def get_financial_indicators(ticker_symbol):
    """
    Retorna um dicionário com indicadores financeiros de uma empresa, dado seu ticker.
    """
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info

    eps = info.get('trailingEps')
    growth = info.get('earningsGrowth')
    graham_price = None
    if eps is not None and growth is not None:
        graham_price = eps * (8.5 + 2 * (growth * 100))  # growth está como fração, ex: 0.1 = 10%


    return {
        'Ticker': ticker_symbol.upper(),
        'Nome': info.get('longName'),
        'Setor': info.get('sector'),
        'Indústria': info.get('industry'),
        'Preço Atual': info.get('currentPrice'),
        'Market Cap': info.get('marketCap'),
        'trailingPE': info.get('trailingPE'),
        'priceToBook': info.get('priceToBook'),
        'returnOnEquity': info.get('returnOnEquity'),
        'returnOnAssets': info.get('returnOnAssets'),
        'debtToEquity': info.get('debtToEquity'),
        'currentRatio': info.get('currentRatio'),
        'profitMargins': info.get('profitMargins'),
        'operatingMargins': info.get('operatingMargins'),
        'dividendYield': info.get('dividendYield'),
        'grahamFairPrice': graham_price,
    }

def evaluate_indicators(indicators):
    """
    Avalia cada indicador com base nos limites definidos em THRESHOLDS.
    Retorna um dicionário com status: 'Bom', 'Ruim' ou 'Dados não disponíveis'.
    """
    avaliacoes = {}
    for key, params in THRESHOLDS.items():
        value = indicators.get(key)
        low, high = params['good']
        label = params['label']

        if value is None:
            status = 'Dados não disponíveis'
        elif key == 'grahamFairPrice':
            preco_atual = indicators.get('Preço Atual')
            if preco_atual is None or value is None:
                status = 'Dados não disponíveis'
            else:
                status = 'Bom' if preco_atual < value else 'Ruim'

        else:
            ok_low = (low is None) or (value >= low)
            ok_high = (high is None) or (value <= high)
            status = 'Bom' if ok_low and ok_high else 'Ruim'

        avaliacoes[label] = status
    return avaliacoes

def compare_companies(tickers):
    """
    Compara várias empresas, mostra indicadores e retorna ranking com base na quantidade de indicadores "Bom".
    """
    resultados = []
    for t in tickers:
        try:
            data = get_financial_indicators(t)
            avals = evaluate_indicators(data)

            print(f"\n=== Indicadores de {data['Ticker']} ({data['Nome']}) ===")
            for key, val in THRESHOLDS.items():
                label = val['label']
                valor = data.get(key)
                status = avals[label]
                print(f"{label}: {valor} -> {status}")

            qtd_bom = sum(1 for s in avals.values() if s == 'Bom')
            total = len([s for s in avals.values() if s != 'Dados não disponíveis'])
            score = qtd_bom / total if total > 0 else 0
            resultados.append({
                'Ticker': data['Ticker'],
                'Nome': data['Nome'],
                'Bom': qtd_bom,
                'Total Avaliados': total,
                'Score': round(score, 2)
            })
        except Exception:
            print(f"Erro ao obter dados para {t.upper()}.")
            resultados.append({'Ticker': t.upper(), 'Nome': None, 'Bom': 0, 'Total Avaliados': 0, 'Score': 0})

    df = pd.DataFrame(resultados)
    df_ranked = df.sort_values(by='Score', ascending=False).reset_index(drop=True)
    return df_ranked

In [7]:
# Definição de limites para avaliação
THRESHOLDS = {
    'trailingPE': {'good': (0, 15), 'label': 'P/L (Price to Earnings)'},
    'priceToBook': {'good': (0, 1.5), 'label': 'P/VPA (Price to Book)'},
    'returnOnEquity': {'good': (0.15, None), 'label': 'ROE (Return on Equity)'},
    'returnOnAssets': {'good': (0.05, None), 'label': 'ROA (Return on Assets)'},
    'debtToEquity': {'good': (None, 1), 'label': 'Dívida/Patrimônio'},
    'currentRatio': {'good': (1.5, None), 'label': 'Current Ratio'},
    'profitMargins': {'good': (0.1, None), 'label': 'Margem Líquida'},
    'operatingMargins': {'good': (0.1, None), 'label': 'Margem Operacional'},
    'dividendYield': {'good': (0.02, None), 'label': 'Dividend Yield'},
    'grahamFairPrice': {'good': (None, None), 'label': 'Preço Justo (Graham)'}
}

In [8]:
if __name__ == '__main__':
    entrada = input('Digite os tickers das empresas separados por vírgula (ex: AAPL, MSFT, GOOGL): ')
    tickers = [t.strip() for t in entrada.split(',') if t.strip()]

    # Compara, mostra indicadores e gera ranking
    ranking = compare_companies(tickers)

    print("\n=== Ranking de Empresas ===")
    print(ranking.to_string(index=False))
    print("\nLegenda: 'Bom' = número de indicadores dentro dos limites; 'Total Avaliados' = indicadores disponíveis; 'Score' = proporção de 'Bom'.")

# Cálculo de desempenho em X meses
    while True:
        try:
            meses = int(input("\nDeseja ver o desempenho dos ativos nos últimos quantos meses? (Digite 0 para sair): "))
            if meses <= 0:
                break
            print(f"\nVariação percentual dos ativos nos últimos {meses} meses:")
            for t in tickers:
                try:
                    hist = yf.Ticker(t).history(period=f"{meses}mo")
                    if len(hist) >= 2:
                        preco_inicio = hist['Close'].iloc[0]
                        preco_fim = hist['Close'].iloc[-1]
                        variacao = (preco_fim - preco_inicio) / preco_inicio * 100
                        print(f"{t}: {variacao:.2f}%")
                    else:
                        print(f"{t}: Dados insuficientes.")
                except Exception:
                    print(f"{t}: Erro ao obter dados.")
        except ValueError:
            print("Entrada inválida. Tente novamente.")

Digite os tickers das empresas separados por vírgula (ex: AAPL, MSFT, GOOGL): AAPL, MSFT, GOOGL

=== Indicadores de AAPL (Apple Inc.) ===
P/L (Price to Earnings): 33.24161 -> Ruim
P/VPA (Price to Book): 49.619312 -> Ruim
ROE (Return on Equity): 1.7142199 -> Bom
ROA (Return on Assets): 0.22964 -> Bom
Dívida/Patrimônio: 152.411 -> Ruim
Current Ratio: 0.893 -> Ruim
Margem Líquida: 0.26915002 -> Bom
Margem Operacional: 0.31647 -> Bom
Dividend Yield: 0.42 -> Bom
Preço Justo (Graham): 1422.2050000000002 -> Bom

=== Indicadores de MSFT (Microsoft Corporation) ===
P/L (Price to Earnings): 31.631765 -> Ruim
P/VPA (Price to Book): 9.093161 -> Ruim
ROE (Return on Equity): 0.32241002 -> Bom
ROA (Return on Assets): 0.14656 -> Bom
Dívida/Patrimônio: 33.154 -> Ruim
Current Ratio: 1.401 -> Ruim
Margem Líquida: 0.35707 -> Bom
Margem Operacional: 0.48873 -> Bom
Dividend Yield: 0.8 -> Bom
Preço Justo (Graham): 475.95599999999996 -> Bom

=== Indicadores de GOOGL (Alphabet Inc.) ===
P/L (Price to Earnings)